# CP 3 Aprendizaje de Máquinas
---
## Árboles de Decisión y Random Forest

### Ejercicio 1: Análisis de dataset Iris

Para demostrar el uso de árboles de decisión y random forest vamos a usar el dataset Iris. 

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

El dataset Iris tiene cuatro características (`sepal length`, `sepal width`, `petal length`, `petal width`) que se pueden usar para clasificar las flores de Iris en tres especies indicadas como "0", "1", "2" (setosa, versicolor, virginica).

In [ ]:
import pandas as pd

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target
df['species_names'] = df.species
df.replace({'species_names':{
            0:iris['target_names'][0],
            1:iris['target_names'][1],
            2:iris['target_names'][2]            
        }}, inplace=True)
df.columns = [item.replace(' (cm)', '') for item in df.columns]

Veamos el formato de este dataset:

In [ ]:
df.head()

Busquemos la cantidad de elementos que pertenecen a cada una de las clases:

In [ ]:
df.species.value_counts()

# import numpy as np

# np.unique(df.species, return_counts=True)

Procedamos a dividir el dataset en un conjunto de entrenamiento y otro de prueba. Se quiere entrenar usando el 70% del dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.7)

### Ejercicio 2: Decision Tree aplicado a Iris

Para probar el funcionamiento de los árboles de decisión, usémoslo en el problema de clasificar las especies de Iris. La implementación por defecto de `sklearn` para árboles de Decisión, `DecisionTreeClassifier` tiene como medida de calidad de la separación de los nodos `gini`, para **Impureza Gini**, para usar la medida dada en clases, **Ganancia de Información**, tenemos que poner el parámetro opcional `criterion` igual a `entropy`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(criterion="entropy")
tree_clf.fit(X_train, y_train)
tree_clf.score(X_test,y_test)

Las reglas del árbol de decisión pueden ser representadas usando un grafo, mediante la función de `sklearn` `plot_tree(decision_tree)` que recibe además como parámetros opcionales, `feature_names` para representar los nombres de las características y `class_names`, los nombres de las clases.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(25,20))
_ = plot_tree(tree_clf, 
                   feature_names=iris.feature_names,  
                   class_names=iris.target_names,
                   filled=True)

Resulta que solo se necesitan dos parámetros, la longitud del pétalo (`petal length`) y el ancho del pétalo (`petal width`), para clasificar la mayoría de los muestras. El parámetro `sepal width`, para el ancho del sépalo, también se usa para hacer las distinciones más finas, pero en última instancia no aporta mucho valor.

### Ejercicio 3: Random Forest aplicado a Iris

Probemos ahora los resultados que se obtienen con Random Forest en el dataset Iris.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest_clf = RandomForestClassifier(criterion="entropy")
forest_clf.fit(X_train, y_train)
forest_clf.score(X_test,y_test)

### Ejercicio 4: Visualizando la Importancia de las Características

Una gran ventaja de los clasificadores basados en árboles es que nos permite hacernos una idea de la importancia relativa de cada carcterística en función de como se dividen los nodos en la fase de entrenamiento. Para ello, tanto el `DecisionTreeClassifier` como el `RandomForestClassifier` de `scikit-learn` proporciona un atributo llamado `feature_importances_`. Esto devuelve un arrray de valores que suman 1. Cuanto mayor sea la puntuación, más importante será la característica. La puntuación se calcula como la reducción total (normalizada) del criterio aportado por esa característica.

In [ ]:
import numpy as np
import seaborn as sns

def plot_feature_importance(feature_imp: pd.Series):
    "Grafica la importancia de cada característica"
    sns.barplot(x=feature_imp, y=feature_imp.index)
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features", pad=15, size=14)

Primero, se grafica la importancia de las características según los resultados obtenidos por el árbol de decisión.

In [ ]:
feature_imp = pd.Series(tree_clf.feature_importances_, 
                        index=['sepal length', 'sepal width', 'petal length', 'petal width']).sort_values(ascending=False)
plot_feature_importance(feature_imp)

Y luego se grafica la importancia de las características del Random Forest.

In [ ]:
feature_imp = pd.Series(forest_clf.feature_importances_, 
                        index=['sepal length', 'sepal width', 'petal length', 'petal width']).sort_values(ascending=False)
plot_feature_importance(feature_imp)

### Ejercicio 5: Visualizar los espacios de decisión

El resultado del árbol de decisión se puede visualizar graficando su espacio de decisión. A continuación se implementa una función (`mesh_plot`) que muestra este resultado utilizando regiones sombreadas que coinciden con los colores utilizados para identificar la flor.

In [ ]:
import seaborn as sns
%matplotlib inline
from matplotlib.colors import ListedColormap

In [ ]:
plt.rcParams.update({'figure.titlesize': 'large'})
step = 0.04

def mesh_plot(x: pd.DataFrame, y: pd.Series, species: pd.Series, ax: plt.Axes, clf):
    values = species.unique()
    colors = sns.color_palette()[:len(values)]
    xx, yy = np.meshgrid(
        np.arange(x.min() - 0.1, x.max() + 0.1, step),
        np.arange(y.min() - 0.1, y.max() + 0.1, step))
    mesh_predict = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    mesh_predict = mesh_predict.reshape(xx.shape)
    for i in range(len(colors)):
        ax.scatter(x[df.species == values[i]], y[df.species == values[i]], color=colors[i])
        ax.set_xlim(x.min() - 0.2, x.max() + 0.2)
        ax.set_ylim(y.min() - 0.2, y.max() + 0.2)
    ax.pcolormesh(xx, yy, mesh_predict,
        cmap=ListedColormap(sns.color_palette()[:3]), alpha=0.2, shading='auto')

La función `plot_features`, dado un dataset representado por `df`, un par de features `feat1` y `feat2`, un tipo de clasificador de árbol especificado por `clsf` y un eje `ax`, grafica el espacio de decisión de dicho clasificador en el eje `ax`.

In [ ]:
def plot_features(df: pd.DataFrame, feat1: str, feat2: str, clsf, ax: plt.Axes):
    """
    Dado un dataset representado por `df`, un par de features `feat1` y `feat2`, un tipo de clasificador de árbol 
    especificado por `clsf` y un eje `ax`, grafica el espacio de decisión de dicho clasificador en el eje `ax`.
    """
    X = df[[feat1, feat2]]
    y = df.species
    fit_clsf = clsf().fit(X, y)
    ax.set(xlabel=feat1, ylabel=feat2)
    mesh_plot(df[feat1], df[feat2], df.species, ax, fit_clsf) 
    

Ahora grafiquemos el espacio de decisión de 4 pares de características del clasificador `DecisionTreeClassifier`.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 8))
# fig.tight_layout()
fig.suptitle('Decision Tree Decision Space')

plot_features(df, 'petal length', 'petal width', DecisionTreeClassifier, ax1)
plot_features(df, 'sepal length', 'sepal width', DecisionTreeClassifier, ax2)
plot_features(df, 'sepal length', 'petal length', DecisionTreeClassifier, ax3)
plot_features(df, 'sepal width', 'petal width', DecisionTreeClassifier, ax4)

Luego, podemos hacer los mismo con `RandomForestClassifier`.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 8))
# fig.tight_layout()
fig.suptitle('Random Forest Decision Space')

plot_features(df, 'petal length', 'petal width', RandomForestClassifier, ax1)
plot_features(df, 'sepal length', 'sepal width', RandomForestClassifier, ax2)
plot_features(df, 'sepal length', 'petal length', RandomForestClassifier, ax3)
plot_features(df, 'sepal width', 'petal width', RandomForestClassifier, ax4)

Se puede notar como los espacios de decisión de los árboles de decisión son más rectos, mientras que el de _Random Forest_ tiene curvas un poco más suaves. 

### Ejercicio 6: Profundidad de los árboles

Utilizando un árbol de decisión con poca profundidad los resultados obtenidos en el dataset no son buenos. A medida que aumenta la profundidad, el árbol de decisiones identifica mejor las especies de Iris. Esto lo podemos comprobar graficando el espacio de decisión para observar los ejemplos que son clasificados mal.

In [ ]:
from typing import List

def plot_dt_by_depth(df: pd.DataFrame, feat1: str, feat2: str, ax: List[plt.Axes]):
  """
  Grafica el espacio de decisión de un árbol de decisión según `feat1` y `feat2`
  usando los datos presentes en un dataframe `df` en los ejes `ax`, que está compuesto
  por 3 ejes.
  """
  # La matriz de características solo está compuesta por las características de interés
  X = df[[feat1, feat2]]
  for idx in range(0, 3):
    # Se crea el árbol de decisión de clasificación con la profundidad determinada, y se realiza el entrenamiento 
    clf = DecisionTreeClassifier(max_depth=idx + 1, random_state=0).fit(X, df.species)
    # Se grafica el espacio de decisión
    mesh_plot(df[feat1], df[feat2], df.species, ax[idx], clf)

Por ejemplo, podemos graficar el árbol teniendo en cuenta dos características: `petal length` y `petal width`.

In [ ]:
# Se crean los subplots
fig, ax = plt.subplots(1, 3, sharey=True, figsize=(15, 5), squeeze=True)
fig.tight_layout()
fig.suptitle('Decision trees with varying depths', y=1.05)

plot_dt_by_depth(df, 'petal length', 'petal width', ax)

### Ejercicio 7: Decision Tree y Random Forest aplicados al dataset de _Rotten Tomatoes_ 

Ahora, probemos los nuevos clasificadores en la tarea de aprendizaje anterior, la clasificación de críticas de _Rotten Tomatoes_.

Como en la clase práctica anterior, extraemos el contenidos de los archivos.

In [ ]:
from pathlib import Path

path_p = Path("txt_sentoken/pos")
path_n = Path("txt_sentoken/neg")

ds_p = list(path_p.iterdir())     # directorio donde están las críticas positivas
ds_n = list(path_n.iterdir())     # directorio donde están las críticas negativas

def convert_file_to_text(file_path: Path) -> str:
    with open(file_path) as f:
        return ''.join(f.readlines())
    
texts_p = [convert_file_to_text(file) for file in ds_p]    # Lista de críticas positivas
texts_n = [convert_file_to_text(file) for file in ds_n]    # Lista de críticas negativas

Y creamos la matriz de características y el vector de clases.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer()
mt = vectorizer.fit_transform(texts_p + texts_n)
mta = mt.toarray()

y = [1]*1000 + [0]*1000

Mediante esta función realizamos los experimentos para comprobar el rendimiento promedio de los algoritmos en varias iteraciones.

In [ ]:
def experiments(Clsf, iterations: int) -> List[float]:
    rs = []
    for _ in range(iterations):
        X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)
        clf = Clsf(criterion="entropy")
        clf.fit(X_train, y_train)
        rs.append(clf.score(X_test, y_test))
    return rs

Ahora probemos los resultados con `DecisionTreeClassifier`.

In [ ]:
results_dt = experiments(DecisionTreeClassifier,30)
np.mean(results_dt)

Y con `RadomForestClassifier`.

In [ ]:
results_rf = experiments(RandomForestClassifier,30)
np.mean(results_rf)